In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tqdm

# Load data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

# Convert DataFrame to numpy array
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

# Convert numpy array to PyTorch tensor
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).unsqueeze(1)

class DNN(nn.Module):
    def __init__(self, input_size, n_layers, n_neurons):
        super(DNN, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, n_neurons[0]))
        layers.append(nn.ReLU())
        for i in range(1, n_layers):
            layers.append(nn.Linear(n_neurons[i-1], n_neurons[i]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(n_neurons[-1], 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def objective(trial):
    # Choose scaler
    scaler_option = trial.suggest_categorical('scaler', ['minmax', 'standard', 'none'])
    if scaler_option == 'minmax':
        scaler = MinMaxScaler()
    elif scaler_option == 'standard':
        scaler = StandardScaler()
    else:
        scaler = None

    # Apply scaler
    if scaler:
        scaler.fit(X_train_np)
        X_train_scaled = scaler.transform(X_train_np)
        X_test_scaled = scaler.transform(X_test_np)
    else:
        X_train_scaled = X_train_np
        X_test_scaled = X_test_np

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).reshape(-1, 1)
    X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32).reshape(-1, 1)

    # Architecture parameters
    n_layers = trial.suggest_int('n_layers', 2, 5)
    n_neurons = [trial.suggest_int(f'n_neurons_{i}', 16, 128) for i in range(n_layers)]
    
    model = DNN(input_size=6, n_layers=n_layers, n_neurons=n_neurons)
    
    # Loss function and optimizer
    loss_fn = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    n_epochs = trial.suggest_int('epochs', 25, 75)
    batch_size = trial.suggest_int('batch_size', 16, 64, step=16)
    batch_start = torch.arange(0, len(X_train_tensor), batch_size)
    
    best_rmse = np.inf

    for epoch in range(n_epochs):
        model.train()
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                X_batch = X_train_tensor[start:start+batch_size]
                y_batch = y_train_tensor[start:start+batch_size]
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                bar.set_postfix(mse=float(loss))

        model.eval()
        y_pred = model(X_test_tensor)
        mse = loss_fn(y_pred, y_test_tensor)
        rmse = np.sqrt(mse.item())
        if rmse < best_rmse:
            best_rmse = rmse

    return best_rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"  RMSE: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Use the best parameters to train the final model
best_scaler_option = trial.params['scaler']
if best_scaler_option == 'minmax':
    best_scaler = MinMaxScaler()
elif best_scaler_option == 'standard':
    best_scaler = StandardScaler()
else:
    best_scaler = None

if best_scaler:
    best_scaler.fit(X_train_np)
    X_train_scaled = best_scaler.transform(X_train_np)
    X_test_scaled = best_scaler.transform(X_test_np)
else:
    X_train_scaled = X_train_np
    X_test_scaled = X_test_np

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32).reshape(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32).reshape(-1, 1)

# Final model with best parameters
n_layers = trial.params['n_layers']
n_neurons = [trial.params[f'n_neurons_{i}'] for i in range(n_layers)]
model = DNN(input_size=6, n_layers=n_layers, n_neurons=n_neurons)

model.eval()
with torch.no_grad():
    for i in range(5):
        X_sample = X_test_tensor[i: i+1]
        y_pred = model(X_sample)
        print(f"{X_test_np[i]} -> {y_pred[0].numpy()} (expected {y_test_np[i]})")


[I 2024-07-23 18:14:38,141] A new study created in memory with name: no-name-e86999aa-a7ef-4313-96ae-b66b1031c1a0
[I 2024-07-23 18:21:17,054] Trial 0 finished with value: 18.072195440700472 and parameters: {'scaler': 'minmax', 'n_layers': 5, 'n_neurons_0': 102, 'n_neurons_1': 82, 'n_neurons_2': 98, 'n_neurons_3': 119, 'n_neurons_4': 68, 'epochs': 52, 'batch_size': 16}. Best is trial 0 with value: 18.072195440700472.
[I 2024-07-23 18:24:13,809] Trial 1 finished with value: 18.62485908608606 and parameters: {'scaler': 'standard', 'n_layers': 3, 'n_neurons_0': 68, 'n_neurons_1': 110, 'n_neurons_2': 77, 'epochs': 34, 'batch_size': 16}. Best is trial 0 with value: 18.072195440700472.
[I 2024-07-23 18:26:23,536] Trial 2 finished with value: 18.018615421529603 and parameters: {'scaler': 'standard', 'n_layers': 3, 'n_neurons_0': 105, 'n_neurons_1': 71, 'n_neurons_2': 104, 'epochs': 71, 'batch_size': 48}. Best is trial 2 with value: 18.018615421529603.
[I 2024-07-23 18:26:58,958] Trial 3 finish

KeyboardInterrupt: 